## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-25-18-52-20 +0000,nyt,Exxon Sues California Over New Climate Disclos...,https://www.nytimes.com/2025/10/25/climate/exx...
1,2025-10-25-18-48-43 +0000,bbc,Connolly declared president of Ireland after l...,https://www.bbc.com/news/articles/c4gk2kml122o...
2,2025-10-25-18-45-14 +0000,wapo,Hurricane Melissa tracker: Map and projected s...,https://www.washingtonpost.com/weather/2025/10...
3,2025-10-25-18-36-04 +0000,nyt,Canada-U.S. Tensions Stay In the Baseball Fiel...,https://www.nytimes.com/2025/10/25/sports/base...
4,2025-10-25-18-32-10 +0000,nypost,"A vestige of Nazi Germany, underground tunnels...",https://nypost.com/2025/10/25/world-news/a-ves...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2515/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
69,trump,49
3,new,13
410,china,10
432,he,10
591,trade,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
206,2025-10-24-21-43-10 +0000,nypost,Ontario to pause anti-tariff Reagan ads after ...,https://nypost.com/2025/10/24/us-news/ontario-...,110
243,2025-10-24-19-57-00 +0000,wsj,Ontario to Pause Anti-Tariff Ad Campaign After...,https://www.wsj.com/world/americas/ontario-to-...,106
254,2025-10-24-19-27-00 +0000,wsj,Ontario to Pause Anti-Tariff Commercial After ...,https://www.wsj.com/world/americas/ontario-to-...,100
56,2025-10-25-14-56-21 +0000,nypost,Trump ‘open’ to meeting with Kim Jong Un durin...,https://nypost.com/2025/10/25/world-news/trump...,99
72,2025-10-25-13-07-30 +0000,bbc,Trump hopes China will help push Russia toward...,https://www.bbc.com/news/articles/cn51yk9vgrko...,93


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
206,110,2025-10-24-21-43-10 +0000,nypost,Ontario to pause anti-tariff Reagan ads after ...,https://nypost.com/2025/10/24/us-news/ontario-...
56,47,2025-10-25-14-56-21 +0000,nypost,Trump ‘open’ to meeting with Kim Jong Un durin...,https://nypost.com/2025/10/25/world-news/trump...
71,41,2025-10-25-13-08-09 +0000,nypost,Kamala Harris teases new White House run: ‘I a...,https://nypost.com/2025/10/25/us-news/kamala-h...
55,38,2025-10-25-15-21-49 +0000,nypost,Chilling footage shows Russian drone hunting d...,https://nypost.com/2025/10/25/world-news/chill...
72,37,2025-10-25-13-07-30 +0000,bbc,Trump hopes China will help push Russia toward...,https://www.bbc.com/news/articles/cn51yk9vgrko...
249,37,2025-10-24-19-35-06 +0000,latimes,Mega Millions alcanza los 680 millones de dóla...,https://www.latimes.com/espanol/california/art...
119,36,2025-10-25-09-00-14 +0000,nypost,"Early voting for NYC mayoral election, other r...",https://nypost.com/2025/10/25/us-news/early-vo...
101,35,2025-10-25-10-00-00 +0000,latimes,Arellano: As Trump blows up supposed narco boa...,https://www.latimes.com/california/story/2025-...
256,33,2025-10-24-19-11-00 +0000,wsj,The Pentagon said it received $130 million fro...,https://www.wsj.com/politics/policy/anonymous-...
190,30,2025-10-24-22-17-00 +0000,wsj,Inflation isn’t as high as many economists exp...,https://www.wsj.com/economy/consumers/inflatio...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
